In [ ]:
from gtsrb_loader.load_data import load_data
from gtsrb_loader.get_folderpath import get_folderpath

import numpy as np

In [ ]:
path = get_folderpath(subset='train', original_images=False)
X_train, y_train = load_data(path)

In [ ]:
y_train



In [ ]:
# normalize X_train
X_train = np.array(X_train, dtype=np.float32) / 255

In [ ]:
# one-hot encode the labels
boundBox=np.array(y_train)[:,0:4]
sign=np.array(y_train)[:,4]
unique_classes = len(np.unique(sign))
y_trainN = np.eye(unique_classes)[np.array(sign, dtype=int)]
y_trainF=np.concatenate([y_trainN,boundBox], axis=1)

In [ ]:
y_trainF



In [ ]:
# Keras Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

## From Keras docs, VGG-like Convnet.

First Block

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(input_shape=(64, 64, 3),
                 filters=32,
                 kernel_size=(3, 3),
                 activation='relu'))

In [ ]:
model.add(Conv2D(filters=32,
                 kernel_size=(3, 3),
                 activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Dropout(0.25))

Second Block

In [ ]:
model.add(Conv2D(filters=64,
                 kernel_size=(3, 3),
                 activation='relu'))

In [ ]:
model.add(Conv2D(filters=64,
                 kernel_size=(3, 3),
                 activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Dropout(0.25))

Fully Connected Layers

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(43, activation='softmax'))

In [ ]:
from keras.optimizers import SGD
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=sgd)

In [ ]:
model.fit(X_train, y_train, batch_size=32, verbose=2)

In [ ]:
model.save('01-07-17_convnet.h5')